# Dividend Payments

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from pathlib import Path
# Sets a variable to the home directory of the current user
home = str(Path.home())

### Using Pandas to open a csv

In [ ]:
# Let's open the file we wrote again, but this time using Pandas
acorns = f'{home}/data/private/acorns2019.csv'
content = pd.read_csv(acorns)

In [ ]:
type(content)

In [ ]:
# Displaying the content that has been saved in a pandas DataFrame
content.head()

In [ ]:
content.head(2)
content = content.drop('Raw', axis=1)
content.head(2)

## Adding a column with the name of the fund for each transaction

In [ ]:
# spaces = content.Description.str.find(" ", 1)
# spaces

# https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html
content = content.assign(Fund=lambda x: x.Description.str.split(' ').str.get(0))

In [ ]:
def convertDollarStringToFloat(df=content, column='Amount'):
    df[column] = df[column].str.replace('$', '')
    df[column] = df[column].astype(float)
    return df

def parseDateTime(df=content, column='Date'):
    df[column] = pd.to_datetime(df[column], format="%m/%d/%Y")
    return df


cleaned_content = parseDateTime(convertDollarStringToFloat(content, 'Amount'), 'Date')

# content['Amount'] = content['Amount'].str.replace('$', '')
# content['Amount'] = content['Amount'].astype(float)
# content['Date'] = pd.to_datetime(content['Date'], format="%m/%d/%Y")
cleaned_content.head()

In [ ]:
dividend_investment_filter = cleaned_content.Description.str.contains("Dividend Reinvestment")
dividend_investments = cleaned_content.loc[dividend_investment_filter, ["Date", "Amount", "Fund"]]
dividend_investments.head()

In [ ]:
dividend_investments.Amount.sum()

In [ ]:
dividend_investments.Amount.count()

In [ ]:
dividend_investments.Amount.mean()

In [ ]:
# converted_records = content.to_dict(orient="records")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
dividends_by_date = dividend_investments.groupby(dividend_investments["Date"].dt.date)['Amount'].agg(['sum']).reset_index()

In [ ]:
# dividends_by_date
# df.groupby(df['date'].dt.year)['a'].agg(['sum', 'mean', 'max'])
fig=plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.plot_date(dividends_by_date['Date'], dividends_by_date['sum'], linestyle='solid')


In [ ]:
dividends_per_month = dividend_investments.resample('m', on='Date')['Amount'].agg(['sum']).reset_index()

In [ ]:
fig=plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(dividends_per_month['Date'], dividends_per_month['sum'])

In [ ]:
# content = content.assign(Fund=lambda x: x.Description.str.split(' ').str.get(1))
# content['Amount'] = content['Amount'].str.replace('$', '')
# content['Amount'] = content['Amount'].astype(float)

dividends_per_month = dividends_per_month.assign(change=lambda x: x['sum'].pct_change().round(3))

In [ ]:
updatedColumns = {"sum":"dividend","Date":"month"}
dividends_per_month = dividends_per_month.rename(columns=updatedColumns)
dividends_per_month

In [ ]:
dividend_investments.head()

In [ ]:
fund_dividends = dividend_investments.groupby(['Fund', 'Date']).agg(['sum']).reset_index()
fund_dividends.columns.droplevel(1)
# .resample('m', on='Date')
fund_dividends.head()

In [ ]:
# Not currently needed - another solution was found shortly after. Leaving for reference.
funds = fund_dividends.Fund.unique().tolist()

import matplotlib.colors as mcolors
colors = mcolors.TABLEAU_COLORS.values()

fund_colors = dict(zip(funds, colors))
fund_colors

In [ ]:
fund_totals = fund_dividends.groupby(['Fund']).sum().reset_index()
fund_totals

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
for key, grp in fund_dividends.groupby('Fund'):
    temp_sum = fund_totals.loc[fund_totals.Fund == key, ['Amount']]
    temp_sum_value = round(float(temp_sum.iloc[0]['Amount']),3)
    
    plt.plot_date(grp['Date'], grp['Amount'], label=f"{key}: {temp_sum_value}", linestyle='solid')

plt.grid()
plt.legend(title="Fund and Total Dividends")
fig.autofmt_xdate()
plt.show()

In [ ]:
# Can be used to validate that the labels above are correct
# fund_totals = fund_dividends.groupby(['Fund']).sum()
# fund_totals